In [ ]:
import numpy as np
import pandas as pd

import folium
import matplotlib.pyplot as plt
import matplotlib
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from folium.plugins import HeatMap
from IPython.display import display

# Libraries for importing datasets from Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# Configuring Google Drive file loading (run it only once)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1MURORv9iCRNNZtZORoNlv3WtDAYcGYqJ'
id = link.split('=')[1]

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile(id)  
df = pd.read_csv(id, sep=',')

Define which dataset to use (whole dataset or just fatal accidents)

In [ ]:
df2 = df[df['Accident_Severity'] == 1]
#df2 = df

Add rows according to the number of casualties in the accident

In [ ]:
df_location = df2[['Longitude','Latitude','Number_of_Casualties']]
latitudes = [] 
longitudes = []
for index, row in df_location.iterrows():
    for i in range(int(row['Number_of_Casualties'])):
        latitudes.append(row['Latitude'])
        longitudes.append(row['Longitude'])

df2 = pd.DataFrame(list(zip(latitudes, longitudes)), columns = ['Latitude', 'Longitude'])

DBSCAN running

In [ ]:
# Change datatypes for using less memory
df_location = df2[['Longitude', 'Latitude']]
df_location = df_location.astype({'Latitude': np.float32, 'Longitude': np.float32})
# Normalizing latitude and longitude
min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = min_max_scaler.fit_transform(df_location)
# DBSCAN running (choose parameters)
accident_location = DBSCAN(eps=0.01, min_samples=50).fit(df_scaled)
#accident_location = DBSCAN(eps=0.001, min_samples=20).fit(df_scaled)
# Get DBSCAN result
df2['labels'] = accident_location.labels_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Plot heatmap and clusters with folium library

In [ ]:
m = folium.Map([df2['Latitude'].iloc[0], df2['Longitude'].iloc[0]], zoom_start=5, control_scale=True)
# Remove noise points
df2 = df2[df2['labels'] != -1]
# Heatmap
HeatMap(data=df2[['Latitude', 'Longitude']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(m)
display(m)

In [ ]:
# Cluster map
m = folium.Map([df2['Latitude'].iloc[0], df2['Longitude'].iloc[0]], zoom_start=5, control_scale=True)
colors = [  'red',
            'blue',
            'gray',
            'darkred',
            'lightred',
            'orange',
            'green',
            'darkgreen',
            'lightgreen',
            'darkblue',
            'lightblue',
            'purple',
            'darkpurple',
            'pink',
            'cadetblue']
colors.sort()

for index, row in df2.iterrows():
    folium.Circle(
        radius=50,
        location=[row['Latitude'], row['Longitude']],
        color=colors[int(row['labels']) % len(colors)],
        fill=True
    ).add_to(m)
display(m)